# Node Classification
AutoGL supports multiple graph related tasks, including node classification. 

In this file we will give you a simple example to show how to use AutoGL to do the node classification task.

## Import libraries
First, you should import some libraries and you can set the random seed before you split the dataset and train the model.

In [ ]:
import yaml
import random
import torch.backends.cudnn
import numpy as np

from autogl.datasets import build_dataset_from_name
from autogl.solver import AutoNodeClassifier
from autogl.module import Acc
from autogl.backend import DependentBackend

# set random seed
random.seed(2022)
np.random.seed(2022)
torch.manual_seed(2022)
if torch.cuda.is_available():
    torch.cuda.manual_seed(2022)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

## Load Dataset
AutoGL provides a very convenient interface to obtain and partition common datasets, such as cora, citeseer, and amazon_computers, etc.

You just need to give the name of the dataset you want and AutoGL will return the dataset.

In this example, we evaluate model on Cora dataset in the semi-supervised node classification task.

In [ ]:
dataset = build_dataset_from_name('cora')

## Initialize Solver
After obtaining the dataset, we need to initialize the model.

However, as AutoGL provides a convenient method to use HPO to better optimize the model, we can train the model through the solver class provided by AutoGL.

Solver in AutoGL usually uses a config file for lazy initialization. The format of the config file can be found in the `../config` folder for examples, or you can read our tutorial for some help.

In [ ]:
if DependentBackend.is_pyg():
    label = dataset[0].y
else:
    label = dataset[0].ndata['label']

configs = yaml.load(open('../../configs/nodeclf_gcn_benchmark_small.yml', "r").read(), Loader=yaml.FullLoader)
autoClassifier = AutoNodeClassifier.from_config(configs)

## Train
After the initialization is finished, you can use the interface provided by AutoGL to optimize the model through HPO.

In [ ]:
# time limit is the seconds limited for training the model
# evaluation method is the metric to evaluate the performance
autoClassifier.fit(dataset, time_limit=3600, evaluation_method=[Acc])

## Evaluation
After training, you can evaluate the model.

In [ ]:
autoClassifier.get_leaderboard().show()
# you can also provided the metric here!
acc = autoClassifier.evaluate(metric="acc")
print("test acc: {:.4f}".format(acc))